We use Shannon Radius as a benchmark, this file shows how we get our deffinitions for that

As we define negative radius to be radius not found, this will allow us to ignore it later

In [50]:
import pandas as pd
from pymatgen.core.periodic_table import Element

In [51]:
#Variables
source_file = 'Data/PbFCl/PbFCl.csv'
periodic_table = 'Data/Periodic Table of Elements.csv'
target_file = 'Data/shannon.csv'

In [52]:
#Get list of elements
with open(source_file,'r') as f:
    els = next(f).split(',')[:-8] # These files have 8 latice parameters in them so we remove them from our list
periodic_table_info = pd.read_csv(periodic_table).set_index("Symbol")

In [53]:
def get_shannon_radius(el):
        if el == 'H' or el == 'D':
            return -1
        el_data = Element(el)._data
    
        try:
            ox_state = str(periodic_table_info.loc[el]['OxidationState'])
            radii = el_data['Shannon radii'][ox_state]['VI']
        except Exception as e:
            print(f'Trouble getting Shannon radius of {el}')
            if 'Shannon radii' in el_data:
                if ox_state in el_data['Shannon radii']:
                    print(f'Invalid coordination state for shannon radius of {el} at oxidation state {ox_state}')
                    first_ox_state = list(el_data['Shannon radii'][ox_state].keys())[0]
                    ox_state = el_data['Shannon radii'][ox_state][first_ox_state]
                    return sum([ox_state[key]['ionic_radius']
                                for key in ox_state]) / len(ox_state)
                
                print(f'Invalid oxidation radius of {el} at oxidation state {ox_state}')
                el_data = Element(el)._data['Shannon radii']
                if len(el_data) == 0:
                    return -1
                return sum([sum([el_data[key]['VI'][key2]['ionic_radius']
                                 for key2 in el_data[key]['VI']]) / len(el_data[key]['VI'])
                            for key in el_data]) / len(el_data)

            return -1
        if len(radii) == 0:
            return -1
        return sum([radii[key]['ionic_radius']for key in radii]) / len(radii)

In [55]:
with open(target_file,'w') as f:
    
    f.write(','.join(els))
    f.write('\n')
    f.write(','.join([str(get_shannon_radius(el)) for el in els]))

Trouble getting Shannon radius of Ar
Trouble getting Shannon radius of Th
Invalid oxidation radius of Th at oxidation state 3
Trouble getting Shannon radius of N
Invalid coordination state for shannon radius of N at oxidation state -3
Trouble getting Shannon radius of Kr
Trouble getting Shannon radius of Xe
Invalid oxidation radius of Xe at oxidation state 3
Trouble getting Shannon radius of Es
Trouble getting Shannon radius of Tc
Invalid oxidation radius of Tc at oxidation state 2
